In [ ]:
from google.colab import drive
drive.mount( '/content/drive' )

Mounted at /content/drive


In [ ]:
pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.8 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error



In [ ]:
from keras.models import Sequential
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import load_model
from keras.initializers import Orthogonal
import datetime
import time

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:

def split_x(size, obs):
    x = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        window = obs[i:(i+SEQUENCE_SIZE)]
        window = [[x] for x in window]
        x.append(window)


    return np.array(x)

In [ ]:
def split_y(size, obs):
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        after_window = obs[i+SEQUENCE_SIZE]
        y.append(after_window)

    return np.array(y)

# **...... BITCOIN......**

In [ ]:
btcpath='/content/BTC-USD (6).csv'


In [ ]:
btc=pd.read_csv(btcpath)

In [ ]:
btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...,...
2474,2023-10-11,27392.076172,27474.115234,26561.099609,26873.320313,26873.320313,13648094333
2475,2023-10-12,26873.292969,26921.439453,26558.320313,26756.798828,26756.798828,9392909295
2476,2023-10-13,26752.878906,27092.697266,26686.322266,26862.375000,26862.375000,15165312851
2477,2023-10-14,26866.203125,26969.000000,26814.585938,26861.707031,26861.707031,5388116782


In [ ]:
btc_features =btc.iloc[:,1:5]

In [ ]:
btc_features

,Open,High,Low,Close
0,963.658020,1003.080017,958.698975,998.325012
1,998.617004,1031.390015,996.702026,1021.750000
2,1021.599976,1044.079956,1021.599976,1043.839966
3,1044.400024,1159.420044,1044.400024,1154.729980
4,1156.729980,1191.099976,910.416992,1013.380005
...,...,...,...,...
2474,27392.076172,27474.115234,26561.099609,26873.320313
2475,26873.292969,26921.439453,26558.320313,26756.798828
2476,26752.878906,27092.697266,26686.322266,26862.375000
2477,26866.203125,26969.000000,26814.585938,26861.707031


In [ ]:
btc_features=scaler.fit_transform(btc_features)

In [ ]:
btc_features

array([[0.00282263, 0.00264503, 0.0030924 , 0.00330246],
       [0.00334617, 0.00306156, 0.00367149, 0.00365319],
       [0.00369036, 0.00324827, 0.00405088, 0.00398393],
       ...,
       [0.38903592, 0.38650601, 0.39512457, 0.39055219],
       [0.39073304, 0.38468603, 0.39707903, 0.39054219],
       [0.39052838, 0.38623614, 0.39720766, 0.39298224]])

In [ ]:
len(btc_features)

2479

In [ ]:
btc_close=btc.iloc[:,4:5]
btc_close=scaler.fit_transform(btc_close)


In [ ]:
#splite X_train and x_test using all features.........

btc_train_size = int(len(btc_features)*.8 )
btc_train=btc_features[:btc_train_size]
btc_test=btc_features[btc_train_size:]


In [ ]:
SEQUENCE_SIZE=30
btcx_train= split_x(SEQUENCE_SIZE,btc_train)
btcx_test = split_x(SEQUENCE_SIZE,btc_test)

print("Shape of training set: {}".format(btcx_train.shape))

Shape of training set: (1953, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

btc_train_size_close = int(len(btc_close)*.8 )
btc_train_close=btc_close[:btc_train_size_close]
btc_test_close=btc_close[btc_train_size_close:]

In [ ]:
btcy_train= split_y(SEQUENCE_SIZE,btc_train_close)
btcy_test = split_y(SEQUENCE_SIZE,btc_test_close)

In [ ]:
btcx_train.shape

(1953, 30, 1, 4)

In [ ]:
btcx_test.shape

(466, 30, 1, 4)

In [ ]:
btcy_train.shape

(1953, 1)

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:
btcx_train = btcx_train.reshape( btcx_train.shape[0], btcx_train.shape[1],4)
btcx_test = btcx_test.reshape( btcx_test.shape[0], btcx_test.shape[1],4)

In [ ]:
btcx_train.shape

(1953, 30, 4)

In [ ]:
btcx_test.shape

(466, 30, 4)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(btcx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=120,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 30, 30)            3240      
                                                                 
 dropout (Dropout)           (None, 30, 30)            0         
                                                                 
 gru_1 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_1 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_2 (GRU)                 (None, 30, 90)            41040     
                                                                 
 dropout_2 (Dropout)         (None, 30, 90)            0         
                                                                 
 gru_3 (GRU)                 (None, 120)               7

In [ ]:
model.fit(btcx_train,btcy_train,epochs=100,batch_size=32)

Epoch 1/100
62/62 [==============================] - 24s 133ms/step - loss: 0.0088
Epoch 2/100
62/62 [==============================] - 5s 79ms/step - loss: 0.0023
Epoch 3/100
62/62 [==============================] - 5s 78ms/step - loss: 0.0021
Epoch 4/100
62/62 [==============================] - 4s 66ms/step - loss: 0.0026
Epoch 5/100
62/62 [==============================] - 5s 78ms/step - loss: 0.0024
Epoch 6/100
62/62 [==============================] - 5s 80ms/step - loss: 0.0028
Epoch 7/100
62/62 [==============================] - 4s 66ms/step - loss: 0.0018
Epoch 8/100
62/62 [==============================] - 5s 76ms/step - loss: 0.0017
Epoch 9/100
62/62 [==============================] - 5s 79ms/step - loss: 0.0019
Epoch 10/100
62/62 [==============================] - 4s 65ms/step - loss: 0.0014
Epoch 11/100
62/62 [==============================] - 5s 73ms/step - loss: 0.0017
Epoch 12/100
62/62 [==============================] - 5s 83ms/step - loss: 0.0018
Epoch 13/100
62/62 [===

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:
btcy_pred=model.predict(btcx_test)

15/15 [==============================] - 2s 18ms/step


In [ ]:
btcy_pred

array([[0.2885552 ],
       [0.30213267],
       [0.30865964],
       [0.3060652 ],
       [0.2960954 ],
       [0.2826088 ],
       [0.2719565 ],
       [0.27803826],
       [0.28727132],
       [0.29308516],
       [0.29754966],
       [0.29579467],
       [0.31076235],
       [0.32654887],
       [0.33116502],
       [0.3253197 ],
       [0.3230582 ],
       [0.31616992],
       [0.3183828 ],
       [0.30433735],
       [0.2962327 ],
       [0.31591272],
       [0.33481613],
       [0.33795148],
       [0.33739907],
       [0.33263415],
       [0.32915327],
       [0.32586494],
       [0.3246718 ],
       [0.32112882],
       [0.3273586 ],
       [0.32720727],
       [0.32783872],
       [0.33656135],
       [0.33070046],
       [0.33528015],
       [0.34296083],
       [0.34381068],
       [0.34850994],
       [0.3469519 ],
       [0.3442639 ],
       [0.3397022 ],
       [0.33376342],
       [0.32991952],
       [0.3020268 ],
       [0.29646385],
       [0.30426812],
       [0.302

In [ ]:
btcy_pred.shape

(466, 1)

In [ ]:
btcy_test.shape

(466, 1)

In [ ]:
y_btc_pred=scaler.inverse_transform(btcy_pred)
y_btc_pred

array([[20050.09 ],
       [20956.916],
       [21392.848],
       [21219.566],
       [20553.693],
       [19652.936],
       [18941.479],
       [19347.674],
       [19964.34 ],
       [20352.64 ],
       [20650.822],
       [20533.607],
       [21533.285],
       [22587.652],
       [22895.959],
       [22505.557],
       [22354.512],
       [21894.451],
       [22042.248],
       [21104.166],
       [20562.863],
       [21877.273],
       [23139.814],
       [23349.22 ],
       [23312.326],
       [22994.082],
       [22761.598],
       [22541.973],
       [22462.285],
       [22225.65 ],
       [22641.732],
       [22631.625],
       [22673.799],
       [23256.377],
       [22864.932],
       [23170.807],
       [23683.791],
       [23740.553],
       [24054.41 ],
       [23950.352],
       [23770.822],
       [23466.15 ],
       [23069.504],
       [22812.773],
       [20949.846],
       [20578.3  ],
       [21099.541],
       [21010.299],
       [21109.096],
       [21166.45 ],


In [ ]:
df_btc = pd.DataFrame(y_btc_pred)
df_btc


,0
0,20050.089844
1,20956.916016
2,21392.847656
3,21219.566406
4,20553.693359
...,...
461,27052.314453
462,26564.240234
463,26387.330078
464,26519.333984


In [ ]:
def process_btc_data(df_btc):
 for x in range(len(df_btc)):
  if x == 0:
    next_day_1= df_btc.iloc[:1,:1]
    print(f"Next day is : {next_day_1}")
    x= x+1
  elif x==2:
    next_day_3 = df_btc.iloc[2:3,:1]
    print(f"Next 3 days is : {next_day_3}")
    x= x+1
  elif x == 6:
    next_day_7 = df_btc.iloc[6:7,:1]
    print(f"Next 7 day is : {next_day_7}")

In [ ]:
process_btc_data(df_btc)

Next day is :               0
0  20050.089844
Next 3 days is :               0
2  21392.847656
Next 7 day is :               0
6  18941.478516


In [ ]:
model.save('btc5.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = load_model('btc5.h5')

In [ ]:
model.save_weights("btc_model.h5")

In [ ]:
import pickle
model_pkl_file = "btc_model.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open(model_pkl_file, 'rb') as file:
    model = pickle.load(file)

In [ ]:
with open("btc_model.pkl", 'rb') as file:
    model = pickle.load(file)

In [ ]:

print('MAE =',mean_absolute_error(btcy_test ,btcy_pred))


MAE = 0.008500032388838721


In [ ]:
print('MAPE =',mean_absolute_percentage_error(btcy_test,btcy_pred))

MAPE = 0.025679496472364307


In [ ]:
print('MSE=',mean_squared_error(btcy_test,btcy_pred))

MSE= 0.0001260198808106197


In [ ]:
#Root Mean Sqquare Erorr
print('RMSE=',np.sqrt(mean_squared_error(btcy_test,btcy_pred)))

RMSE= 0.011225857687082074


# **......LETCOIN......**

In [ ]:
ltcpath='/content/LTC-USD (1).csv'


In [ ]:
ltc=pd.read_csv(ltcpath)

In [ ]:
ltc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-18,5.065430,5.065430,4.579960,4.685230,4.685230,4569260
1,2014-09-19,4.687290,4.755820,4.254350,4.327770,4.327770,3917450
2,2014-09-20,4.329200,4.616080,4.202190,4.286440,4.286440,5490660
3,2014-09-21,4.263070,4.300130,4.154990,4.245920,4.245920,2931220
4,2014-09-22,4.245930,4.416880,4.210130,4.242350,4.242350,1855960
...,...,...,...,...,...,...,...
3310,2023-10-11,63.619778,63.685921,61.262772,61.499821,61.499821,186389594
3311,2023-10-12,61.501141,61.521873,60.754681,61.127815,61.127815,171888213
3312,2023-10-13,61.127197,61.995136,61.112534,61.574059,61.574059,203849754
3313,2023-10-14,61.574585,61.949982,61.470036,61.665157,61.665157,120785909


In [ ]:
ltc_features =ltc.iloc[:,1:5]

In [ ]:
ltc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3315 entries, 0 to 3314
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3315 non-null   object 
 1   Open       3315 non-null   float64
 2   High       3315 non-null   float64
 3   Low        3315 non-null   float64
 4   Close      3315 non-null   float64
 5   Adj Close  3315 non-null   float64
 6   Volume     3315 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 181.4+ KB


In [ ]:
ltc_features=scaler.fit_transform(ltc_features)

In [ ]:
ltc_features

array([[0.01011644, 0.00903907, 0.0100708 , 0.00915722],
       [0.00913862, 0.00828689, 0.00912477, 0.00822946],
       [0.00821264, 0.0079474 , 0.00897322, 0.00812219],
       ...,
       [0.1550853 , 0.1473471 , 0.17432131, 0.15680775],
       [0.15624219, 0.1472374 , 0.17536   , 0.15704419],
       [0.15652653, 0.14716836, 0.17531297, 0.15729666]])

In [ ]:
ltc_close=ltc.iloc[:,4:5]
ltc_close=scaler.fit_transform(ltc_close)


In [ ]:
#splite X_train and x_test using all features.........

ltc_train_size = int(len(ltc_features)*.8 )
ltc_train=ltc_features[:ltc_train_size]
ltc_test=ltc_features[ltc_train_size:]


In [ ]:
SEQUENCE_SIZE=30
ltcx_train= split_x(SEQUENCE_SIZE,ltc_train)
ltcx_test = split_x(SEQUENCE_SIZE,ltc_test)


print("Shape of training set: {}".format(ltcx_train.shape))

Shape of training set: (2622, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

ltc_train_size_close = int(len(ltc_close)*.8 )
ltc_train_close=ltc_close[:ltc_train_size_close]
ltc_test_close=ltc_close[ltc_train_size_close:]

In [ ]:
ltcy_train= split_y(SEQUENCE_SIZE,ltc_train_close)
ltcy_test = split_y(SEQUENCE_SIZE,ltc_test_close)

In [ ]:
ltcx_train.shape


(2622, 30, 1, 4)

In [ ]:
ltcx_test.shape

(633, 30, 1, 4)

In [ ]:
ltcy_train.shape

(2622, 1)

In [ ]:
ltcy_test.shape

(633, 1)

In [ ]:
ltcx_train = ltcx_train.reshape( ltcx_train.shape[0], ltcx_train.shape[1], 4)
ltcx_test = ltcx_test.reshape( ltcx_test.shape[0], ltcx_test.shape[1], 4)

In [ ]:
ltcx_test.shape

(633, 30, 4)

In [ ]:
ltcx_train.shape

(2622, 30, 4)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(ltcx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 30, 30)            3240      
                                                                 
 dropout_4 (Dropout)         (None, 30, 30)            0         
                                                                 
 gru_5 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_5 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_6 (GRU)                 (None, 90)                41040     
                                                                 
 dropout_6 (Dropout)         (None, 90)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [ ]:
model.fit(ltcx_train,ltcy_train,epochs=10,batch_size=30)

Epoch 1/10
88/88 [==============================] - 11s 42ms/step - loss: 0.0036
Epoch 2/10
88/88 [==============================] - 4s 43ms/step - loss: 0.0017
Epoch 3/10
88/88 [==============================] - 5s 62ms/step - loss: 0.0015
Epoch 4/10
88/88 [==============================] - 4s 43ms/step - loss: 0.0014
Epoch 5/10
88/88 [==============================] - 4s 44ms/step - loss: 0.0015
Epoch 6/10
88/88 [==============================] - 5s 58ms/step - loss: 0.0013
Epoch 7/10
88/88 [==============================] - 4s 46ms/step - loss: 0.0011
Epoch 8/10
88/88 [==============================] - 4s 45ms/step - loss: 0.0011
Epoch 9/10
88/88 [==============================] - 4s 50ms/step - loss: 0.0011
Epoch 10/10
88/88 [==============================] - 5s 52ms/step - loss: 0.0011


In [ ]:
ltcy_pred=model.predict(ltcx_test)

20/20 [==============================] - 1s 13ms/step


In [ ]:
y_ltc_pred=scaler.inverse_transform(ltcy_pred)

In [ ]:
y_ltc_pred

array([[140.66107 ],
       [131.79279 ],
       [121.41011 ],
       [115.80513 ],
       [113.295334],
       [112.276146],
       [112.22469 ],
       [111.57448 ],
       [111.705025],
       [112.844765],
       [113.41316 ],
       [113.23835 ],
       [114.878426],
       [115.44291 ],
       [114.43066 ],
       [116.88877 ],
       [121.25209 ],
       [125.605835],
       [131.73946 ],
       [136.41502 ],
       [139.33305 ],
       [140.63358 ],
       [137.13385 ],
       [132.63683 ],
       [130.39546 ],
       [129.0161  ],
       [130.26315 ],
       [131.651   ],
       [128.63593 ],
       [123.935814],
       [120.71198 ],
       [118.0239  ],
       [114.191475],
       [111.13219 ],
       [110.42556 ],
       [108.80648 ],
       [109.41808 ],
       [111.247025],
       [110.77869 ],
       [111.679924],
       [114.24378 ],
       [115.23585 ],
       [115.18545 ],
       [112.59555 ],
       [109.5954  ],
       [107.82215 ],
       [105.77268 ],
       [104.5

In [ ]:
df_ltc = pd.DataFrame(y_ltc_pred)
df_ltc

,0
0,140.661072
1,131.792786
2,121.410110
3,115.805130
4,113.295334
...,...
628,68.479988
629,67.614662
630,66.597649
631,66.055290


In [ ]:
def process_ltc_data(df_ltc):
  for x in range(len(df_ltc)):
    if x == 0:
       next_day_1= df_ltc.iloc[:1,:1]
       print(f"Next day is : {next_day_1}")
       x= x+1
    elif x==2:
        next_day_3 = df_ltc.iloc[2:3,:1]
        print(f"Next 3 days is : {next_day_3}")
        x= x+1
    elif x == 6:
        next_day_7 = df_ltc.iloc[6:7,:1]
        print(f"Next 7 day is : {next_day_7}")

In [ ]:
process_ltc_data(df_ltc)

Next day is :             0
0  140.661072
Next 3 days is :            0
2  121.41011
Next 7 day is :             0
6  112.224693


In [ ]:
model.save('ltc.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = load_model('ltc.h5')

In [ ]:
model.save_weights("ltc_model.h5")

In [ ]:
import pickle
model_pkl_file = "ltc_model.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open("ltc_model.pkl", 'rb') as file:
    model = pickle.load(file)

In [ ]:
with open(model_pkl_file, 'rb') as file:
    model = pickle.load(file)

In [ ]:

print('MAE =',mean_absolute_error(ltcy_test,ltcy_pred))


MAE = 0.013235394722530458


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ltcy_test,ltcy_pred))

MAPE = 0.06960066840426307


In [ ]:
print('MSE=',mean_squared_error(ltcy_test,ltcy_pred))

MSE= 0.00026022730778134867


In [ ]:
#Root Mean Squre Erorr
print('RMSE=',np.sqrt(mean_squared_error(ltcy_test,ltcy_pred)))

RMSE= 0.01613156247179264


# **......ETHEREUM......**

In [ ]:
ethpath='/content/ETH-USD.csv'


In [ ]:
eth=pd.read_csv(ethpath)

In [ ]:
eth

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-12-01,445.209015,472.609009,428.312012,466.540009,466.540009,1247879936
1,2017-12-02,466.851013,476.239014,456.653015,463.449005,463.449005,943649984
2,2017-12-03,463.704987,482.813995,451.851990,465.852997,465.852997,990556992
3,2017-12-04,466.053986,474.777008,453.312012,470.204010,470.204010,1005550016
4,2017-12-05,470.294006,473.558014,457.660004,463.281006,463.281006,1216720000
...,...,...,...,...,...,...,...
2140,2023-10-11,1567.680786,1578.223511,1548.980469,1566.254761,1566.254761,5416504273
2141,2023-10-12,1566.355957,1566.878174,1523.237549,1539.612427,1539.612427,5003930677
2142,2023-10-13,1539.432861,1571.750610,1537.921143,1552.089478,1552.089478,4575141511
2143,2023-10-14,1552.263794,1560.325073,1545.738770,1555.256836,1555.256836,2429214718


In [ ]:
eth_features=eth.iloc[: , 1:5]

In [ ]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2145 non-null   object 
 1   Open       2145 non-null   float64
 2   High       2145 non-null   float64
 3   Low        2145 non-null   float64
 4   Close      2145 non-null   float64
 5   Adj Close  2145 non-null   float64
 6   Volume     2145 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 117.4+ KB


In [ ]:

eth_features=scaler.fit_transform(eth_features)

In [ ]:
eth_close=eth.iloc[:,4:5]
eth_close=scaler.fit_transform(eth_close)


In [ ]:
#splite X_train and x_test using all features.........

eth_train_size = int(len(eth_features)*.8 )

eth_train=eth_features[:eth_train_size]
eth_test=eth_features[eth_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ethx_train= split_x(SEQUENCE_SIZE,eth_train)
ethx_test = split_x(SEQUENCE_SIZE,eth_test)

print("Shape of training set: {}".format(ethx_train.shape))

Shape of training set: (1686, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

eth_train_size_close = int(len(eth_close)*.8 )

eth_train_close=eth_close[:eth_train_size_close]
eth_test_close=eth_close[eth_train_size_close:]

In [ ]:
ethy_train= split_y(SEQUENCE_SIZE,eth_train_close)
ethy_test = split_y(SEQUENCE_SIZE,eth_test_close)

In [ ]:
ethx_train.shape


(1686, 30, 1, 4)

In [ ]:
ethx_test.shape

(399, 30, 1, 4)

In [ ]:
ethy_train.shape

(1686, 1)

In [ ]:
ethy_test.shape

(399, 1)

In [ ]:
ethx_train = ethx_train.reshape( ethx_train.shape[0], ethx_train.shape[1], 4)
ethx_test = ethx_test.reshape( ethx_test.shape[0], ethx_test.shape[1], 4)

In [ ]:
ethx_test.shape

(399, 30, 4)

In [ ]:
ethx_train.shape

(1686, 30, 4)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(ethx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_7 (GRU)                 (None, 30, 30)            3240      
                                                                 
 dropout_7 (Dropout)         (None, 30, 30)            0         
                                                                 
 gru_8 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_8 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_9 (GRU)                 (None, 90)                41040     
                                                                 
 dropout_9 (Dropout)         (None, 90)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                

In [ ]:
model.fit(ethx_train,ethy_train,epochs=10,batch_size=30)

Epoch 1/10
57/57 [==============================] - 9s 43ms/step - loss: 0.0101
Epoch 2/10
57/57 [==============================] - 3s 45ms/step - loss: 0.0025
Epoch 3/10
57/57 [==============================] - 3s 59ms/step - loss: 0.0023
Epoch 4/10
57/57 [==============================] - 3s 54ms/step - loss: 0.0020
Epoch 5/10
57/57 [==============================] - 3s 45ms/step - loss: 0.0018
Epoch 6/10
57/57 [==============================] - 2s 42ms/step - loss: 0.0019
Epoch 7/10
57/57 [==============================] - 2s 43ms/step - loss: 0.0018
Epoch 8/10
57/57 [==============================] - 3s 60ms/step - loss: 0.0018
Epoch 9/10
57/57 [==============================] - 3s 53ms/step - loss: 0.0018
Epoch 10/10
57/57 [==============================] - 2s 43ms/step - loss: 0.0015


In [ ]:
ethy_pred=model.predict(ethx_test)

13/13 [==============================] - 1s 12ms/step


In [ ]:
y_eth_pred=scaler.inverse_transform(ethy_pred)
y_eth_pred

array([[1760.7501],
       [1783.1265],
       [1769.7925],
       [1742.0493],
       [1695.1389],
       [1633.7444],
       [1583.2134],
       [1531.3427],
       [1485.2188],
       [1446.414 ],
       [1405.7155],
       [1376.6455],
       [1363.3108],
       [1359.5845],
       [1356.6367],
       [1358.1266],
       [1363.6266],
       [1368.6011],
       [1372.2985],
       [1375.024 ],
       [1374.2164],
       [1366.1317],
       [1360.7306],
       [1366.2886],
       [1376.4945],
       [1386.6355],
       [1391.3799],
       [1388.872 ],
       [1383.9641],
       [1375.5009],
       [1363.2334],
       [1353.5253],
       [1345.1622],
       [1340.8934],
       [1336.7078],
       [1336.8209],
       [1344.2981],
       [1351.8425],
       [1352.8383],
       [1349.0911],
       [1345.8538],
       [1347.057 ],
       [1358.0741],
       [1371.1031],
       [1397.9912],
       [1446.4905],
       [1494.0795],
       [1534.2952],
       [1575.3276],
       [1608.3667],


In [ ]:
df_eth = pd.DataFrame(y_eth_pred)
df_eth

,0
0,1760.750122
1,1783.126465
2,1769.792480
3,1742.049316
4,1695.138916
...,...
394,1677.366821
395,1660.041870
396,1642.733521
397,1629.639526


In [ ]:
def process_eth_data(df_eth):
    for x in range(len(df_eth)):
       if x == 0:
         next_day_1= df_eth.iloc[:1,:1]
         print(f"Next day is : {next_day_1}")
         x= x+1
       elif x==2:
          next_day_3 = df_eth.iloc[2:3,:1]
          print(f"Next 3 days is : {next_day_3}")
          x= x+1
       elif x == 6:
           next_day_7 = df_eth.iloc[6:7,:1]
           print(f"Next 7 day is : {next_day_7}")

In [ ]:
process_eth_data(df_eth)

Next day is :              0
0  1760.750122
Next 3 days is :             0
2  1769.79248
Next 7 day is :              0
6  1583.213379


In [ ]:
model.save('eth.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = load_model('eth.h5')

In [ ]:
model.save_weights("eth_model.h5")

In [ ]:
print('MAE =',mean_absolute_error(ethy_test,ethy_pred))


MAE = 0.016070290340750734


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ethy_test,ethy_pred))

MAPE = 0.050754367547781114


In [ ]:
print('MSE=',mean_squared_error(ethy_test,ethy_pred))

MSE= 0.00038880068316948275


In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(ethy_test,ethy_pred)))

RMSE= 0.019718029393666164


# **......XMR.....**

In [ ]:
xmr_path='/content/XMR-USD.csv'

In [ ]:
xmr=pd.read_csv(xmr_path)

In [ ]:
xmr

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000
1,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000
2,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000
3,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000
4,2017-11-14,123.615997,124.200996,118.625999,122.352997,122.352997,53544800
...,...,...,...,...,...,...,...
2161,2023-10-11,152.220093,153.929443,150.970428,153.842102,153.842102,61162383
2162,2023-10-12,153.852783,154.344238,151.021347,152.341812,152.341812,65445853
2163,2023-10-13,152.339523,154.368774,151.051971,153.590515,153.590515,51282283
2164,2023-10-14,153.583801,154.282990,151.850967,152.833160,152.833160,33344438


In [ ]:
xmr_features=xmr.iloc[: , 1:5]

In [ ]:
xmr_features

,Open,High,Low,Close
0,121.344002,121.665001,101.757004,105.585999
1,105.750000,127.106003,103.877998,119.615997
2,119.597000,133.675003,110.617996,123.856003
3,128.960007,136.528000,120.921997,123.402000
4,123.615997,124.200996,118.625999,122.352997
...,...,...,...,...
2161,152.220093,153.929443,150.970428,153.842102
2162,153.852783,154.344238,151.021347,152.341812
2163,152.339523,154.368774,151.051971,153.590515
2164,153.583801,154.282990,151.850967,152.833160


In [ ]:
xmr_features=scaler.fit_transform(xmr_features)

In [ ]:
xmr_features

array([[0.19585978, 0.17676522, 0.17589732, 0.16107407],
       [0.16125454, 0.18807766, 0.18086816, 0.19221218],
       [0.19198294, 0.20173534, 0.19666427, 0.20162242],
       ...,
       [0.26464313, 0.24475998, 0.29142686, 0.26761504],
       [0.26740435, 0.24458163, 0.29329942, 0.26593417],
       [0.26562223, 0.24288277, 0.29405913, 0.26592798]])

In [ ]:
xmr_close=xmr.iloc[:,4:5]
xmr_close=scaler.fit_transform(xmr_close)


In [ ]:
#splite X_train and x_test using all features.........

xmr_train_size = int(len(xmr_features)*.8 )

xmr_train=xmr_features[:xmr_train_size]
xmr_test=xmr_features[xmr_train_size:]

In [ ]:
SEQUENCE_SIZE=30
xmrx_train= split_x(SEQUENCE_SIZE,xmr_train)
xmrx_test = split_x(SEQUENCE_SIZE,xmr_test)

print("Shape of training set: {}".format(xmrx_train.shape))

Shape of training set: (1702, 30, 1, 4)


In [ ]:
# splite y_train and y_test using Close only.......

xmr_train_size_close = int(len(xmr_close)*.8 )

xmr_train_close=xmr_close[:xmr_train_size_close]
xmr_test_close=xmr_close[xmr_train_size_close:]

In [ ]:
xmry_train= split_y(SEQUENCE_SIZE,xmr_train_close)
xmry_test = split_y(SEQUENCE_SIZE,xmr_test_close)

In [ ]:
xmrx_train.shape


(1702, 30, 1, 4)

In [ ]:
xmrx_test.shape

(404, 30, 1, 4)

In [ ]:
xmry_train.shape

(1702, 1)

In [ ]:
xmry_test.shape

(404, 1)

In [ ]:
xmrx_train = xmrx_train.reshape( xmrx_train.shape[0], xmrx_train.shape[1], 4)
xmrx_test = xmrx_test.reshape( xmrx_test.shape[0], xmrx_test.shape[1], 4)

In [ ]:
xmrx_test.shape

(404, 30, 4)

In [ ]:
xmrx_train.shape

(1702, 30, 4)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(xmrx_train.shape[1],4)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, 30, 30)            3240      
                                                                 
 dropout_10 (Dropout)        (None, 30, 30)            0         
                                                                 
 gru_11 (GRU)                (None, 30, 60)            16560     
                                                                 
 dropout_11 (Dropout)        (None, 30, 60)            0         
                                                                 
 gru_12 (GRU)                (None, 90)                41040     
                                                                 
 dropout_12 (Dropout)        (None, 90)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [ ]:
model.fit(xmrx_train,xmry_train,epochs=10,batch_size=30)

Epoch 1/10
57/57 [==============================] - 9s 43ms/step - loss: 0.0103
Epoch 2/10
57/57 [==============================] - 2s 43ms/step - loss: 0.0029
Epoch 3/10
57/57 [==============================] - 2s 44ms/step - loss: 0.0027
Epoch 4/10
57/57 [==============================] - 3s 55ms/step - loss: 0.0025
Epoch 5/10
57/57 [==============================] - 3s 58ms/step - loss: 0.0020
Epoch 6/10
57/57 [==============================] - 3s 44ms/step - loss: 0.0020
Epoch 7/10
57/57 [==============================] - 2s 42ms/step - loss: 0.0020
Epoch 8/10
57/57 [==============================] - 2s 43ms/step - loss: 0.0021
Epoch 9/10
57/57 [==============================] - 3s 56ms/step - loss: 0.0022
Epoch 10/10
57/57 [==============================] - 3s 56ms/step - loss: 0.0019


In [ ]:
xmry_pred=model.predict(xmrx_test)

In [ ]:
y_xmr_pred=scaler.inverse_transform(xmry_pred)
y_xmr_pred

array([[146.50626 ],
       [143.77493 ],
       [143.12756 ],
       [145.07951 ],
       [147.97942 ],
       [149.76715 ],
       [151.87016 ],
       [149.63226 ],
       [146.33781 ],
       [143.42432 ],
       [141.8343  ],
       [141.58156 ],
       [139.51753 ],
       [136.72087 ],
       [135.44626 ],
       [133.7323  ],
       [133.03772 ],
       [133.49446 ],
       [134.37189 ],
       [134.92043 ],
       [135.71907 ],
       [136.97891 ],
       [137.7959  ],
       [139.25685 ],
       [140.38448 ],
       [139.4412  ],
       [136.71681 ],
       [134.80405 ],
       [135.41704 ],
       [137.02913 ],
       [138.7102  ],
       [139.65915 ],
       [140.18782 ],
       [140.29242 ],
       [139.34142 ],
       [138.22949 ],
       [137.93532 ],
       [136.73227 ],
       [135.8566  ],
       [135.01006 ],
       [134.89629 ],
       [135.68655 ],
       [136.86765 ],
       [138.0333  ],
       [137.6815  ],
       [136.2966  ],
       [135.63693 ],
       [136.0

In [ ]:
df_xmr = pd.DataFrame(y_xmr_pred)
df_xmr

,0
0,146.506256
1,143.774933
2,143.127563
3,145.079514
4,147.979416
...,...
399,147.229416
400,146.893448
401,146.520355
402,146.351273


In [ ]:
def process_xmr_data(df_xmr):
    for x in range(len(df_xmr)):
        if x == 0:
          next_day_1= df_xmr.iloc[:1,:1]
          print(f"Next day is : {next_day_1}")
          x= x+1
        elif x==2:
          next_day_3 = df_xmr.iloc[2:3,:1]
          print(f"Next 3 days is : {next_day_3}")
          x= x+1
        elif x == 6:
          next_day_7 = df_xmr.iloc[6:7,:1]
          print(f"Next 7 day is : {next_day_7}")

In [ ]:
process_xmr_data(df_xmr)

Next day is :             0
0  146.506256
Next 3 days is :             0
2  143.127563
Next 7 day is :             0
6  151.870163


In [ ]:
model.save('xmr.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model = load_model('xmr.h5')

In [ ]:
model.save_weights("xmr_model.h5")

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE =',mean_absolute_error(xmry_test,xmry_pred))


MAE = 0.016173551683496774


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print('MAPE =',mean_absolute_percentage_error(xmry_test,xmry_pred))

MAPE = 0.06119591666038092


In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE=',mean_squared_error(xmry_test,xmry_pred))

MSE= 0.00032963158197501857


In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(xmry_test,xmry_pred)))